# geodesyNET vs masconCUBE
A geodesyNET is but a new representation of the mass density of an irregular body.

One can see it as a parametrization of a continuous function $\rho(x,y,z)$ whose parameters $\eta_i$ (network weights and biases) are learned efficiently thanks to the SGD approach.

The question arises on how it compares with respect to the state-of-the-art in geodesy techniques. While any comparison to existing methods is bound to be unfair as geodesyNETs are the only representation that can learn the body shape and its interior structure simultaneously, we can introduce an alternative model (we call masconCUBE) that can be compared in fairness to GeodesyNET.

A masconCUBE is a cube full of mascons so that $N\times N\times N$ mascons $m_j$ are placed withing the unit volume $V$ in a regular grid. We may then consider the various masses $m_j$ as the parameters of a model to be learned from observations.

Differently from a geodesyNET, a masconCUBE does not represent the body density continuously, but other than this important detail the two representations have similar properties and hence can be compared once the various parameters $\eta_i$ for the network and $m_i$ for the masconCUBE are learned.

![alt text](figures/masconCUBE.png)

In this notebook we setup the learning procedure for a masconCUBE. Note that the value of the gravitational acceleration created by a masconCUBE at $\mathbf r_i$ is:
$$
\mathbf a_i = \sum_{j=1}^{N^3} \frac {m_j}{r_{ij}^3}{\mathbf r_{ij}}
$$
which means that each measurment of a gravitational acceleration results in linear relation so that at the end putting all measurements together one may write:
$$
\mathbf A \mathbf m = \mathbf b
$$
In most cases, when $N>20$ this system becomes to large to be solved and one must revert to alternative approaches, in particular gradient descent based.

In [ ]:
# Import our module containing helper functions
import gravann

# Core imports
import numpy as np
import scipy
import pickle as pk
import os
from collections import deque
from copy import deepcopy

# pytorch
from torch import nn
import torch

# plotting stuff
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

# If possible enable CUDA
gravann.enableCUDA()
gravann.fixRandomSeeds()
device = os.environ["TORCH_DEVICE"]
print("Will use device ",device)

# Loading and visualizing a mascon model
For the purpose of this notebook we will be using, as ground truth, also a mascon model: the one of Eros.
Note though that the procedure in generic and one could also use a polyhedral gravity gound truth or other models.

In [ ]:
target = "models/Churyumov-Gerasimenko.mdl"
target = target.split(".")[0].split("/")[1]

# We load the ground truth (a mascon model of some body)
with open("mascons/"+target+".pk", "rb") as file:
    mascon_points, mascon_masses, mascon_name = pk.load(file)
    
mascon_points = torch.tensor(mascon_points)
mascon_masses = torch.tensor(mascon_masses)

# Print some information on the loaded ground truth 
# (non-dimensional units assumed. All mascon coordinates are thus in -1,1 and the mass is 1)
print("Name: ", mascon_name)
print("Number of mascons: ", len(mascon_points))
print("Total mass: ", sum(mascon_masses))

In [ ]:
# Here we visualize the loaded ground truth
gravann.plot_mascon(mascon_points, mascon_masses)

# Learning the masconCUBE model


In [ ]:
# This is the number of mascons per side of our masconCUBE
N=23
print("MasconCUBE number of parameters is: ", N*N*N)
# Here we define the sqrt(m_j) or model parameters
mascon_masses_model = torch.ones((N*N*N,1), requires_grad=True)
#mascon_masses_model = mascon_masses_model.clone().detach().requires_grad_(True)

In [ ]:
# Here we define the points of the masconCUBE
X,Y,Z = torch.meshgrid(torch.linspace(-1,1, N), torch.linspace(-1,1, N), torch.linspace(-1,1, N), indexing='ij')
X = X.reshape(-1,1)
Y = Y.reshape(-1,1)
Z = Z.reshape(-1,1)
mascon_points_model = torch.concat((X,Y,Z), dim=1)

In [ ]:
# Dimension of the batch size, i.e. number of points
# where the ground truth is compared to the predicted acceleration
# at each training epoch.
batch_size = 1000

# Loss. The normalized L1 loss. 
loss_fn = gravann.normalized_L1_loss

# The sampling method to decide what points to consider in each batch.
# In this case we sample points unifromly in a sphere and reject those that are inside the asteroid
targets_point_sampler = gravann.get_target_point_sampler(batch_size, 
                                                         limit_shape_to_asteroid="3dmeshes/"+target+"_lp.pk", 
                                                         method="spherical", 
                                                         bounds=[0,1])

# Here we set the optimizer
learning_rate = 1e-1
optimizer = torch.optim.Adam(params = [mascon_masses_model], lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.8, patience = 200, min_lr = 1e-8,verbose=True)

# And init the best results
best_loss = np.inf

# When a new network is created we init empty training logs
loss_log = []
weighted_average_log = []
running_loss_log = []
# .. and we init a loss trend indicators
weighted_average = deque([], maxlen=20)

In [ ]:
# TRAINING LOOP (normal training, no use of any prior shape information)------------------------
# This cell can be stopped and started again without loosing memory of the training nor its logs
torch.cuda.empty_cache()
# The main training loop
for i in range(3000):
    # Each ten epochs we resample the target points
    if (i % 10 == 0):
        target_points = targets_point_sampler()
        # We compute the labels whenever the target points are changed
        labels = gravann.ACC_L(target_points, mascon_points, mascon_masses)
    
    # We compute the values predicted by the neural density field
    predicted = gravann.ACC_L(target_points, mascon_points_model, mascon_masses_model*mascon_masses_model)
    
    # We compute the scaling constant (k in the paper) used in the loss
    c = torch.sum(predicted*labels)/torch.sum(predicted*predicted)
    
    # We compute the loss
    loss = loss_fn(predicted.view(-1), labels.view(-1))
    
    # We store the model if it has the lowest fitness 
    # (this is to avoid losing good results during a run that goes wild)
    if loss < best_loss:
        best_model = deepcopy(mascon_masses_model)
        best_loss = loss
        print('New Best: ', loss.item())
    
    # Update the loss trend indicators
    weighted_average.append(loss.item())
    
    # Update the logs
    weighted_average_log.append(np.mean(weighted_average))
    loss_log.append(loss.item())
    
    # Print every i iterations
    if i % 25 == 0:
        wa_out = np.mean(weighted_average)
        print(f"It={i}\t loss={loss.item():.3e}\t  weighted_average={wa_out:.3e}\t  c={c:.3e}")
        
    # Zeroes the gradient (necessary because of things)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()
    
    # Perform a step in LR scheduler to update LR
    scheduler.step(loss.item())

In [ ]:
plt.figure()
plt.semilogy(loss_log)

In [ ]:
# The actual value of the mascon masses is here obtained by squaring and normalizing
final_mascon_masses = best_model*best_model/torch.sum(best_model*best_model)

In [ ]:
torch.save(final_mascon_masses,"cube_masses_"+target+".tensor")
torch.save(mascon_points_model,"cube_points_"+target+".tensor")

In [ ]:
final_mascon_masses = torch.load("cube_masses_"+target+".tensor").detach()
mascon_points_model = torch.load("cube_points_"+target+".tensor").detach()

In [ ]:
# Units of Length (this is Eros) (depend on how the the mascon model was created .... )
# These numbers matter for the final values quantitatively (but not qualitatively)
L = {
    "Bennu.pk": 352.1486930549145,
    "Bennu_nu.pk": 352.1486930549145,
    "Churyumov-Gerasimenko.pk": 3126.6064453124995, 
    "Eros.pk" : 20413.864850997925,
    "Itokawa.pk": 350.438691675663,
    "Itokawa_nu.pk": 350.438691675663,
    "Torus.pk": 3126.6064453124995,
    "Hollow.pk": 3126.6064453124995,
    "Hollow_nu.pk": 3126.6064453124995,
    "Hollow2.pk": 3126.6064453124995,
    "Hollow2_nu.pk": 3126.6064453124995
}

R0 = {
    "Eros.pk" : 16000,
    "Itokawa.pk": 300,
    "Churyumov-Gerasimenko.pk": 4300,
    "Bennu.pk": 565
}
L = L[target + ".pk"]
R0 = R0[target + ".pk"] / L

In [ ]:
# Let us generate all Stokes coefficients up to order 7 and degree 7 (a square model)
l=7
m=7
stokesC_mascon, stokesS_mascon = gravann.mascon2stokes(mascon_points_model.cpu().numpy(), final_mascon_masses.view(-1).detach().cpu().numpy(), R0, l, m)

In [ ]:
# Here we first trim the small values (for visualization purposes)
final_mascon_masses[final_mascon_masses<1e-5] = 0
# Then we plot
gravann.plot_mascon(mascon_points_model, final_mascon_masses.detach())

# Quantitative comparison with ground truth and geodestNET

In [ ]:
with open("stokes/stokes_77_"+target+"_gt.pk", "rb") as file:
    stokesC_gt, stokesS_gt = pk.load(file)
with open("stokes/stokes_77_"+target+"_gann.pk", "rb") as file:
    stokesC_gann, stokesS_gann = pk.load(file)

In [ ]:
# Here we compute the sorted indexes corresponding to the largest values
idxs = np.dstack(np.unravel_index(np.argsort(-np.abs(stokesC_gt.ravel())), (8, 8)))[0]
# We print to screen the three models
print(f"Harmonics \tGround Truth \tGeodesyNET \tMasconCUBE")
abs_errs_gann = []
abs_errs_mascon = []
rel_errs_gann = []
rel_errs_mascon = []

N_largest = 30

for n,pos in enumerate(idxs):
    if n == 0:
        continue
    if n == N_largest+1:
        break
    abs_errs_gann.append(abs(stokesC_gt[l][m] - stokesC_gann[l][m]))
    abs_errs_mascon.append(abs(stokesC_gt[l][m] - stokesC_mascon[l][m]))
    if(stokesC_gt[l][m] > 0):
        rel_errs_gann.append(abs(stokesC_gt[l][m] - stokesC_gann[l][m]) / stokesC_gt[l][m])
        rel_errs_mascon.append(abs(stokesC_gt[l][m] - stokesC_mascon[l][m]) / stokesC_gt[l][m])
    l,m = pos
    print(f"C_{l}{m}, \t\t{stokesC_gt[l][m]:2.3e}, \t{stokesC_gann[l][m]:2.3e}, \t{stokesC_mascon[l][m]:2.3e}")

print("geodesyNet Mean Abs. Err. =",np.mean(abs_errs_gann))
print("Mascon Mean Abs. Err. =",np.mean(abs_errs_mascon))
print("geodesyNet Mean Rel. Err. =",np.mean(rel_errs_gann))
print("Mascon Mean Rel. Err. =",np.mean(rel_errs_mascon))

In [ ]:
gravann.validation_mascon(mascon_points_model,final_mascon_masses,
                          mascon_points,mascon_masses, 
                          N=10000,asteroid_pk_path="3dmeshes/"+target+".pk",
#                           sampling_altitudes=[0.005,0.01,0.025],
                         )

In [ ]:
# Encoding: direct encoding (i.e. feeding the network directly with the Cartesian coordinates in the unit hypercube)
# was found to work well in most cases. But more options are implemented in the module.
encoding = gravann.direct_encoding()

# The model is here a SIREN network (FFNN with sin non linearities and a final absolute value to predict the density)
model = gravann.init_network(encoding, hidden_layers=3,n_neurons=100, model_type="siren", activation = gravann.AbsLayer())
# Uncomment to run on CPU
#model.load_state_dict(torch.load("models/eros.mdl", map_location=torch.device('cpu')))
model.load_state_dict(torch.load("models/"+target+".mdl"))

In [ ]:
# Computes the Validation table with rel and abs errors on the predicted acceleration (w.r.t. ground truth) 
# at low, med, high altitudes (see paper). is requires sampling quite a lot, so it takes time 
gravann.validation(model, encoding, mascon_points, mascon_masses, use_acc=True, 
                   asteroid_pk_path="3dmeshes/"+target+".pk", N_integration=500000, N=10000, progressbar=True,
                   #sampling_altitudes=[0.005,0.01,0.025]
                  )

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)